In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
import threading
import time
import pandas as pd

In [2]:
class TradingApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self,self)
        self.pos_df = pd.DataFrame(
            columns = ['Account','Symbol','SecType','Currency','Position','Avg cost']
        )
    def nextValidId(self, orderId):
        super().nextValidId(orderId)
        self.nextValidOrderId = orderId
        print(f"NextValidId: {orderId}")
    def position(self,account, contract, position, avgCost):
        super().position(account, contract, position, avgCost)
        dictionary = {
            "Account":account,
            "Symbol":contract.symbol,
            "SecType":contract.secType,
            "Currency":contract.currency,
            "Position":position,
            "Avg cost":avgCost
        }
        if self.pos_df["Symbol"].str.contains(contract.symbol).any():
            self.pos_df.loc[self.pos_df["Symbol"] == contract.symbol,"Position"] = position
            self.pos_df.loc[self.pos_df["Symbol"] == contract.symbol,"Avg cost"] = avgCost
        else:
            self.pos_df = pd.concat((self.pos_df, pd.DataFrame([dictionary])),ignore_index=True)

def websocket_con():
    app.run()

In [4]:
app = TradingApp()
app.connect("127.0.0.1", 7497, clientId = 12)

In [ ]:
con_thread = threading.Thread(target = websocket_con, daemon =True)
con_thread.start()
time.sleep(1)

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


NextValidId: 1


C:\Users\TunKedsaro\AppData\Local\Temp\ipykernel_15060\1410910564.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.pos_df = pd.concat((self.pos_df, pd.DataFrame([dictionary])),ignore_index=True)
ERROR 1 10311 This order will be directly routed to NASDAQ. Direct routed orders may result in higher trade fees. Restriction is specified in Precautionary Settings of Global Configuration/API.


In [6]:
def usStk(symbol, sectype = "STK", currecy = "USD", exchange = "NASDAQ"):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = sectype
    contract.currency = currecy
    contract.exchange = exchange
    return contract
def mktOrder(direction,quantity):
    order = Order()
    order.action = direction
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.eTradeOnly = ""
    order.firmQuoteOnly = ""
    return order

In [7]:
app.reqPositions()
time.sleep(1)
pos_df = app.pos_df

In [8]:
app.placeOrder(app.nextValidOrderId,usStk("QCOM"),mktOrder("BUY",4))

In [9]:
app.reqPositions()
time.sleep(1)
pos_df = app.pos_df

In [10]:
pos_df

,Account,Symbol,SecType,Currency,Position,Avg cost
0,U21833425,IBKR,STK,USD,0.0282,62.056738


In [11]:
app.disconnect()